<a href="https://colab.research.google.com/github/seobando/UDACITY_GenerativeAI/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#!pip install -q "datasets==2.15.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [48]:
#!pip install -U transformer

In [49]:
#!pip install peft

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: gpt2
* Evaluation approach: accuracy evaluation
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [50]:
dataset_name= "imdb"
model_name = "gpt2"

In [51]:
# Import the datasets and transformers packages

from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset(dataset_name, split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(800))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 800
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 800
 })}

In [52]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    """Preprocess the emotion dataset by returning tokenized examples."""
    return tokenizer(
        examples["text"],
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=128
    )

In [53]:
tokenized_data = {}

for split in splits:
    tokenized_data[split] = ds[split].map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [54]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

model.config.pad_token_id = tokenizer.pad_token_id

for param in model.base_model.parameters():
    param.requires_grad = False

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [56]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [57]:
from transformers import Trainer, DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args= training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [58]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [59]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.695367,0.588750


TrainOutput(global_step=200, training_loss=0.7278119659423828, metrics={'train_runtime': 146.8059, 'train_samples_per_second': 5.449, 'train_steps_per_second': 1.362, 'total_flos': 418074800947200.0, 'train_loss': 0.7278119659423828, 'epoch': 1.0})

In [60]:
trainer.evaluate()

{'eval_loss': 0.6953672766685486,
 'eval_accuracy': 0.58875,
 'eval_runtime': 69.6483,
 'eval_samples_per_second': 11.486,
 'eval_steps_per_second': 2.872,
 'epoch': 1.0}

In [61]:
trainer.save_model("gpt2_model")

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [62]:
gpt_model = AutoModelForSequenceClassification.from_pretrained("gpt2_model")

In [63]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    inference_mode=False,
    r=18,
    lora_alpha=32,
    lora_dropout=0.1,
    modules_to_save=["classifier"],
    bias='lora_only',
)

lora_model = get_peft_model(gpt_model, config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [64]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],
    lora_dropout=0.1,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.SEQ_CLS
)

lora_model = get_peft_model(model, config)

In [65]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    label_names=["labels"],
    load_best_model_at_end=True,
)

In [66]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [67]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.654813,0.637500
2,No log,0.583715,0.702500
3,No log,0.554680,0.721250


Checkpoint destination directory ./results/checkpoint-200 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=300, training_loss=0.5043776448567708, metrics={'train_runtime': 71.3687, 'train_samples_per_second': 33.628, 'train_steps_per_second': 4.204, 'total_flos': 158275731456000.0, 'train_loss': 0.5043776448567708, 'epoch': 3.0})

In [68]:
trainer.evaluate()

{'eval_loss': 0.5546796917915344,
 'eval_accuracy': 0.72125,
 'eval_runtime': 6.9409,
 'eval_samples_per_second': 115.258,
 'eval_steps_per_second': 14.407,
 'epoch': 3.0}

In [69]:
lora_model.save_pretrained("gpt-lora")

In [70]:
tokenizer.save_pretrained("gpt-lora")

('gpt-lora/tokenizer_config.json',
 'gpt-lora/special_tokens_map.json',
 'gpt-lora/vocab.json',
 'gpt-lora/merges.txt',
 'gpt-lora/added_tokens.json',
 'gpt-lora/tokenizer.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.